In [1]:
library("edgeR")
setwd('~/Documents/experiment/')

Loading required package: limma



In [2]:
# Loading counts and metadata tables
countsData <- read.table('output/featureCounts/merged_gene_counts.txt', sep = '\t', row.names = 1, header = T) # counts table of two samples from different cell lines
metaData <- read.table('SraRunTable.txt', header = T, sep = ',') # Metadata table of samples
head(countsData)
metaData

,gene_name,SRR925742_1,SRR1283038_1
,<chr>,<int>,<int>
ENSG00000223972.5,DDX11L1,0,2
ENSG00000227232.5,WASH7P,9,25
ENSG00000278267.1,MIR6859-1,0,0
ENSG00000243485.5,MIR1302-2HG,0,0
ENSG00000284332.1,MIR1302-2,0,0
ENSG00000237613.2,FAM138A,0,0


Run,Assay.Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Cell_Line,Cell_type,Center.Name,⋯,LibrarySelection,LibrarySource,Organism,Platform,ReleaseDate,Sample.Name,source_name,SRA.Study,subtype,quality..run.
<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
SRR925742,RNA-Seq,152,4253695832,PRJNA210428,SAMN02225564,2139823332,ZR75B,breast cancer,GEO,⋯,cDNA,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2015-07-22T00:00:00Z,GSM1172910,breast cancer cell line,SRP026537,Luminal,NA
SRR1283038,RNA-Seq,154,6907369604,PRJNA210428,SAMN02768721,3756629102,MDAMB453,breast cancer cell line,GEO,⋯,cDNA,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2014-05-12T00:00:00Z,GSM1384316,breast cancer cell line,SRP026537,Luminal,NA


In [3]:
# Preproccessing
colnames(countsData)[c(2,3)] <- gsub("_1", "", colnames(countsData)[c(2,3)]) # Gene name formatting: removing sufix _1 from sample names
names(countsData)
cellLine <- as.factor(metaData$Cell_Line) # groups to be compared: 2 different cell lines 
names(cellLine) <- metaData$Run
geneName <- countsData$gene_name
countsData$gene_name <- NULL # countsData only with counts columns of each sample

[1] "gene_name"  "SRR925742"  "SRR1283038"

In [4]:
# Differential expression functions
dgeList <-DGEList(counts = countsData, group = cellLine, genes = geneName) 
sapply(dgeList, dim) # Number of genes before filtering

counts,samples,genes
29985,2,29985
2,3,1


In [5]:
# Filtering out genes with low amount of counts
toKeep <- filterByExpr(dgeList)
dgeListFilt <- dgeList[toKeep, , keep.lib.sizes=FALSE]
sapply(dgeListFilt, dim) # Number of genes after filtering

counts,samples,genes
1079,2,1079
2,3,1


In [6]:
# Data Normalization. by default with TMM method
dgeListNorm <- calcNormFactors(dgeListFilt)
sapply(dgeListNorm, head)
dgeListNorm <- estimateCommonDisp(dgeListNorm)
dgeListNorm$common.dispersion <- 0.1 # Dispersion is setting manually due to the lack of replicates. Is needed for computing foldchange

$counts
                  SRR925742 SRR1283038
ENSG00000227232.5         9         25
ENSG00000279457.4        19         35
ENSG00000225972.1      1028        192
ENSG00000225630.1      3497       1018
ENSG00000237973.1     12303      12954
ENSG00000229344.1      4057       3849

$samples
              group lib.size norm.factors
SRR925742     ZR75B    93866    0.8958021
SRR1283038 MDAMB453    99903    1.1163180

$genes
                     genes
ENSG00000227232.5   WASH7P
ENSG00000279457.4   WASH9P
ENSG00000225972.1 MTND1P23
ENSG00000225630.1 MTND2P28
ENSG00000237973.1 MTCO1P12
ENSG00000229344.1 MTCO2P12

Warning message in estimateCommonDisp.DGEList(dgeListNorm):
“There is no replication, setting dispersion to NA.”


In [9]:
# Computing fold change 
dgeListFC <- exactTest(dgeListNorm) 
# Top differential expressed genes
(topDEG <- topTags(dgeListFC))

,genes,logFC,logCPM,PValue,FDR
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000137975.8,CLCA2,-9.524991,12.256025,6.785266e-17,7.321302e-14
ENSG00000185499.16,MUC1,9.146239,11.508080,1.792863e-15,9.672495e-13
ENSG00000117519.16,CNN3,-10.612446,10.012702,4.322283e-12,1.554581e-09
ENSG00000162383.13,SLC1A7,-10.404146,9.811095,1.637838e-11,4.418067e-09
ENSG00000188460.4,ACTBP11,6.240111,9.536169,8.449521e-10,1.823407e-07
ENSG00000178715.8,AL450998.1,5.577043,9.438242,9.239616e-09,1.661591e-06
ENSG00000230364.1,RPL4P3,5.122906,9.702888,2.223249e-08,3.426979e-06
ENSG00000225616.2,AL138785.1,9.053653,8.444698,6.691571e-08,8.379931e-06
ENSG00000230777.1,RPS29P5,5.232967,9.107470,6.989748e-08,8.379931e-06
